# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [6]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fauske,NO,67.2588,15.3918,41.04,100,100,2.89,light rain
1,1,Thompson,CA,55.7435,-97.8558,21.36,93,75,3.44,broken clouds
2,2,Olonkinbyen,SJ,70.9221,-8.7187,24.82,91,100,29.44,snow
3,3,Kenai,US,60.5544,-151.2583,35.65,88,100,6.91,moderate rain
4,4,Adamstown,PN,-25.0660,-130.1015,74.43,71,53,7.54,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [7]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [8]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]


# Display sample data
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Adamstown,PN,-25.0660,-130.1015,74.43,71,53,7.54,broken clouds
5,5,San Patricio,US,28.0170,-97.5169,75.06,98,2,6.62,clear sky
17,17,San Luis De La Loma,MX,17.2706,-100.8939,73.96,67,50,3.71,scattered clouds
19,19,Grajau,BR,-5.8194,-46.1386,78.84,86,95,2.44,overcast clouds
21,21,Gondar,ET,12.6000,37.4667,71.64,46,99,6.35,light rain


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [9]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()
clean_travel_cities_df = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Adamstown,PN,-25.0660,-130.1015,74.43,71,53,7.54,broken clouds
5,5,San Patricio,US,28.0170,-97.5169,75.06,98,2,6.62,clear sky
17,17,San Luis De La Loma,MX,17.2706,-100.8939,73.96,67,50,3.71,scattered clouds
19,19,Grajau,BR,-5.8194,-46.1386,78.84,86,95,2.44,overcast clouds
21,21,Gondar,ET,12.6000,37.4667,71.64,46,99,6.35,light rain
22,22,Tolanaro,MG,-25.0319,46.9987,78.87,73,40,16.11,scattered clouds
23,23,Port Elizabeth,ZA,-33.9180,25.5701,75.09,68,40,9.22,scattered clouds
25,25,Wailua Homesteads,US,22.0669,-159.3780,70.59,87,100,13.80,overcast clouds
32,32,Waingapu,ID,-9.6567,120.2641,79.74,83,34,7.43,scattered clouds
40,40,Caiaponia,BR,-16.9567,-51.8103,73.65,76,28,1.48,scattered clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [11]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Adamstown,PN,74.43,broken clouds,-25.0660,-130.1015,
5,San Patricio,US,75.06,clear sky,28.0170,-97.5169,
17,San Luis De La Loma,MX,73.96,scattered clouds,17.2706,-100.8939,
19,Grajau,BR,78.84,overcast clouds,-5.8194,-46.1386,
21,Gondar,ET,71.64,light rain,12.6000,37.4667,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [12]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [13]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
       hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Adamstown - nearest hotel: No hotel found
San Patricio - nearest hotel: No hotel found
San Luis De La Loma - nearest hotel: No hotel found
Grajau - nearest hotel: Torre Palace Hotel
Gondar - nearest hotel: Hotel Dashen
Tolanaro - nearest hotel: Hôtel Mahavokey
Port Elizabeth - nearest hotel: Waterford Hotel
Wailua Homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
Waingapu - nearest hotel: HOTEL SANDLEWOOD
Caiaponia - nearest hotel: No hotel found
Matoury - nearest hotel: La Marmotte
Tsiombe - nearest hotel: No hotel found
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Maralal - nearest hotel: Peacock Rest House
Port Mathurin - nearest hotel: Escale Vacances
Kingston - nearest hotel: Aphrodite Hotel
Avarua - nearest hotel: Paradise Inn
Guayaramerin - nearest hotel: Hotel K d' Oro
Afaahiti - nearest hotel: Omati Lodge
Huarmey - nearest hotel: Hostal Santa Rosa
El Jadid - nearest hotel: Hotel El Jadida فندق الجديدة
Boa Vista - nearest

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Adamstown,PN,74.43,broken clouds,-25.0660,-130.1015,No hotel found
5,San Patricio,US,75.06,clear sky,28.0170,-97.5169,No hotel found
17,San Luis De La Loma,MX,73.96,scattered clouds,17.2706,-100.8939,No hotel found
19,Grajau,BR,78.84,overcast clouds,-5.8194,-46.1386,Torre Palace Hotel
21,Gondar,ET,71.64,light rain,12.6000,37.4667,Hotel Dashen
...,...,...,...,...,...,...,...
654,Kirakira,SB,79.59,few clouds,-10.4544,161.9205,No hotel found
656,Korla,CN,71.22,clear sky,41.7597,86.1469,Silver Star Hotel
657,Nicoya,CR,78.12,overcast clouds,10.1483,-85.4520,Hotel marianela
660,Lazaro Cardenas,MX,70.48,scattered clouds,17.9583,-102.2000,Hotel Sol del Pacífico


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [14]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
19,Grajau,BR,78.84,overcast clouds,-5.8194,-46.1386,Torre Palace Hotel
21,Gondar,ET,71.64,light rain,12.6000,37.4667,Hotel Dashen
22,Tolanaro,MG,78.87,scattered clouds,-25.0319,46.9987,Hôtel Mahavokey
23,Port Elizabeth,ZA,75.09,scattered clouds,-33.9180,25.5701,Waterford Hotel
25,Wailua Homesteads,US,70.59,overcast clouds,22.0669,-159.3780,Hilton Garden Inn Kauai Wailua Bay
32,Waingapu,ID,79.74,scattered clouds,-9.6567,120.2641,HOTEL SANDLEWOOD
44,Matoury,GF,75.16,broken clouds,4.8483,-52.3317,La Marmotte
57,Puerto Ayora,EC,75.54,broken clouds,-0.7393,-90.3518,Hostal La Mirada De Solitario George
64,Maralal,KE,71.44,light rain,1.0968,36.6980,Peacock Rest House
65,Port Mathurin,MU,79.16,few clouds,-19.6833,63.4167,Escale Vacances


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [15]:
# Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [16]:
# Configure the map plot

hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    alpha= 0.7,
    
    #by adding hover_cols, we can see the info associated with each city, 
    hover_cols = ["City","Hotel Name","Country","Current Description"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)